In [7]:
from constants import *
import os
import subprocess 
import json

import dotenv


from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account 
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))

# Loading EV
load_dotenv

# Loading Mnemonic EV and (set this mnemonic as an environment variable, and include the one you generated as a fallback using)
mnemonic = os.getenv('MNEMONIC')
print(mnemonic)

def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys
 

# Create a coin object to hold child wallets
coins = {"eth", "btc-test", "btc"}
numderive = 3
# Setting the dictionarry
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

# Creating a private keys object
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(keys, indent=4, sort_keys=True))

#2 - convert private key string 
def priv_key_to_account(coin, priv_key):
    """Convert the privkey string in a child key to an account object that bit or web3.py can use to transact"""
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

#3 create transaction with necesseary metadata

def create_trx(coin, account, recipient, amount):
    """create the raw, unsigned transaction that contains all metadata needed to transact"""
    global trx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)]) 

#4 create, sign, send the transaction 

def send_trx(coin, account, recipient, amount):
    """call create_trx, sign the transaction, then send it to the designated network"""
    if coin == "eth": 
        trx_eth = create_trx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest= create_trx(coin,account,recipient,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest

None
"0xd36d0f9ff2f6dc80208cd19e0b7d939f11bd1a63d10c38cdc1c58f4dbbc95edc"
"cUMT7ptJgxrwSm9s5zSUCKUhSNVq1UC9v31vgoSusyukAciNMdDt"
{
    "btc": [
        {
            "address": "1Hk26uv3aSo2WmKNPbeUGjNWPFDBLU2waW",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "KzQ3o74J1R3MFuzvRNQWjckRz3AYZHgggJRbs3P7TH1ZoXJqKety",
            "pubkey": "0399c9a8f6933934bfe811fc524c0e8684f971819398a0581f20ddf6371db0bb18",
            "pubkeyhash": "b7a4d19972d97725ebe1d980ac875f4b41322ec5",
            "xprv": "xprvA4EtnVRNTKMCodewrh6ipRZg96w7tyJMntxXmtPitFkMn7Dwe6BRfeC2bnL8DRs96stL1Kn752eZJqfFX1Fs25uaRxnUSSShU91qzeg5gWK",
            "xpub": "xpub6HEFBzxGHguW27jQxidjBZWQh8mcJS2DA7t8aGoLSbHLeuZ6BdVgDSWWT5kbpKZxGjjA8qDfgYCyKgBeau5Qp48c5KC9dTaYjWd3jLfF8MG"
        },
        {
            "address": "16LsmsW5nshaMWz4XXE9qZjE1HHEEF61Ma",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L3PRGxRUzyYGDEnWsdtyunpa5DCfYcgKfBSCCXGEHof

In [8]:
coins[BTC][0]['privkey']

TypeError: 'set' object is not subscriptable